In [1]:
!pip install -q transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.3 MB/s eta 0:00:00


In [2]:
!pip install pyarrow

In [3]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00


In [4]:
!pip -q install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00


## Load data

In [5]:
import wandb

In [6]:
from urllib import request
import pandas as pd
from collections import Counter
import random
import numpy as np
import pandas as pd
import copy
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

In [7]:
import datasets
from datasets import Dataset
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer

In [8]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [9]:
from dont_patronize_me import DontPatronizeMe

In [27]:
def set_seeds(seed=42):
    """
    Set random seeds for NumPy and PyTorch.

    Parameters:
    - seed (int): The seed value to set for random number generators.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [11]:
dpm = DontPatronizeMe('.', 'task4_test.tsv')

In [12]:
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [44]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [45]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

In [76]:
set_seeds()
data=dpm.train_task1_df
data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [77]:
set_seeds()
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

trdf1 = pd.DataFrame(rows)
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [82]:
set_seeds()
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })
tedf1 = pd.DataFrame(rows)
tedf1

,par_id,community,text,label
0,4046,hopeless,We also know that they can benefit by receivin...,1
1,1279,refugee,Pope Francis washed and kissed the feet of Mus...,1
2,8330,refugee,Many refugees do n't want to be resettled anyw...,1
3,4063,in-need,"""Budding chefs , like """" Fred """" , """" Winston ...",1
4,4089,homeless,"""In a 90-degree view of his constituency , one...",1
...,...,...,...,...
2089,10462,homeless,"The sad spectacle , which occurred on Saturday...",0
2090,10463,refugee,""""""" The Pakistani police came to our house and...",0
2091,10464,disabled,"""When Marie O'Donoghue went looking for a spec...",0
2092,10465,women,"""Sri Lankan norms and culture inhibit women fr...",0


In [83]:
rows_list = tedf1.values.tolist()

In [84]:
random.seed(42)
# Shuffle the list using random.shuffle
random.shuffle(rows_list)

In [85]:
# Create a new DataFrame from the shuffled list
tedf1 = pd.DataFrame(rows_list, columns=tedf1.columns)
tedf1

,par_id,community,text,label
0,9830,hopeless,Since Mr. Cruz 's election to the Senate in 20...,0
1,9165,refugee,These are some of the dilemmas that befell Min...,0
2,9265,immigrant,"Lucy Lawless ( Salem , Spartacus ) as Ruby , a...",0
3,8457,refugee,The situation has some parallels to Myanmar 's...,0
4,10122,poor-families,"Five years down the lane , the schools whose l...",0
...,...,...,...,...
2089,9170,migrant,"This May 5 , 2018 , photo , released by the Ro...",0
2090,9264,vulnerable,"""Asim Qureshi , of Cageprisoners , says that h...",0
2091,9396,immigrant,Mr Ariel told ministry of housing staff and of...,0
2092,2953,poor-families,""""""" The worst thing was dealt to us , """" he sa...",1


In [86]:
import datasets
from datasets import Dataset
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer

In [87]:
# Split the dataset into train and test sets
train_df, val_df = train_test_split(trdf1, test_size=0.2, random_state=42)

train_dataset =  Dataset.from_dict({"data": train_df["text"], "label": train_df["label"]}).with_format("torch")
val_dataset =  Dataset.from_dict({"data": val_df["text"], "label": val_df["label"]}).with_format("torch")
test_dataset =  Dataset.from_dict({"data": tedf1["text"], "label": tedf1["label"]}).with_format("torch")

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

6700
1675
2094


In [88]:
def tokenize_function(example):
    return tokenizer(example["data"], truncation=True)


In [89]:
from sklearn.metrics import accuracy_score, f1_score

# Define evaluation metrics function
def compute_metrics(p):
    predictions = p.predictions.argmax(axis=1)
    labels = p.label_ids

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='binary')


    # return {"accuracy": accuracy, "f1": f1, "predictions": list(predictions)}
    return {"accuracy": accuracy, "f1": f1}

In [90]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [95]:
tedf1.to_csv('tedf1_fi.csv')
trdf1.to_csv('trdf1_fi.csv')

## Roberta Hate speech

In [91]:
# Load model

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [92]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/6700 [00:00<?, ? examples/s]

Map:   0%|          | 0/1675 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

In [93]:
batch_size = 16

training_args = TrainingArguments("train_logs", evaluation_strategy="epoch",\
                                  learning_rate=2e-6,
                                  num_train_epochs=10,
                                  lr_scheduler_type="cosine",
                                  optim = "adamw_torch",
                                  weight_decay=0,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  run_name="roberta-base-hate-best",  # name of the W&B run (optional)
                                  logging_steps=1,  # how often to log to W&B
                                  )


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [94]:
set_seeds()
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.307000,0.250863,0.894925,0.022222
2,0.073900,0.250973,0.910448,0.364407
3,0.017000,0.251704,0.914627,0.416327
4,0.334900,0.258194,0.909851,0.501650
5,0.008100,0.276570,0.908060,0.503226
6,0.062200,0.307459,0.904478,0.529412
7,0.036900,0.321634,0.908657,0.540541
8,0.002900,0.321871,0.907463,0.525994
9,0.429100,0.324030,0.906866,0.512500
10,0.053800,0.324712,0.906866,0.509434


TrainOutput(global_step=4190, training_loss=0.1726585432560892, metrics={'train_runtime': 624.5745, 'train_samples_per_second': 107.273, 'train_steps_per_second': 6.709, 'total_flos': 4431694616515200.0, 'train_loss': 0.1726585432560892, 'epoch': 10.0})

In [96]:
predictions_val_roberta = trainer.predict(tokenized_val_dataset).predictions
print(predictions_val_roberta.shape)
predictions_val_roberta_lables = predictions_val_roberta.argmax(axis=1)
predictions_val_roberta_lables

(1675, 2)


array([0, 0, 0, ..., 0, 0, 0])

In [97]:
labels2file([[k] for k in predictions_val_roberta_lables], 'val_roberta.txt')

In [98]:
trainer.predict(tokenized_val_dataset).metrics

{'test_loss': 0.3247120678424835,
 'test_accuracy': 0.906865671641791,
 'test_f1': 0.5094339622641509,
 'test_runtime': 3.7746,
 'test_samples_per_second': 443.76,
 'test_steps_per_second': 27.818}

In [99]:
results = trainer.evaluate(tokenized_val_dataset)

print(results)

{'eval_loss': 0.3247120678424835, 'eval_accuracy': 0.906865671641791, 'eval_f1': 0.5094339622641509, 'eval_runtime': 3.837, 'eval_samples_per_second': 436.538, 'eval_steps_per_second': 27.365, 'epoch': 10.0}


In [100]:
predictions_test_roberta = trainer.predict(tokenized_test_dataset)
print(predictions_test_roberta.metrics)

predictions_test_roberta_labels = predictions_test_roberta.predictions.argmax(axis=1)
predictions_test_roberta

{'test_loss': 0.2672642767429352, 'test_accuracy': 0.9255014326647565, 'test_f1': 0.5958549222797928, 'test_runtime': 5.0303, 'test_samples_per_second': 416.276, 'test_steps_per_second': 26.042}


PredictionOutput(predictions=array([[ 3.6689665, -3.4222057],
       [ 3.62206  , -3.421681 ],
       [ 3.3639572, -3.1571689],
       ...,
       [ 3.5532362, -3.4409027],
       [-2.5206122,  2.1157298],
       [ 3.6119652, -3.446675 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 1, 0]), metrics={'test_loss': 0.2672642767429352, 'test_accuracy': 0.9255014326647565, 'test_f1': 0.5958549222797928, 'test_runtime': 5.0303, 'test_samples_per_second': 416.276, 'test_steps_per_second': 26.042})

In [101]:
labels2file([[k] for k in predictions_test_roberta_labels], 'dev_roberta.txt')

In [102]:
Counter(predictions_test_roberta_labels)

Counter({0: 1907, 1: 187})

In [103]:
dpm.load_test()
test_df = dpm.test_set_df

In [110]:
test_df

,par_id,art_id,keyword,country,text
0,t_0,@@7258997,vulnerable,us,"In the meantime , conservatives are working to..."
1,t_1,@@16397324,women,pk,In most poor households with no education chil...
2,t_2,@@16257812,migrant,ca,The real question is not whether immigration i...
3,t_3,@@3509652,migrant,gb,"In total , the country 's immigrant population..."
4,t_4,@@477506,vulnerable,ca,"Members of the church , which is part of Ken C..."
...,...,...,...,...,...
3827,t_3893,@@20319448,migrant,jm,In a letter dated Thursday to European Commiss...
3828,t_3894,@@9990672,poor-families,au,They discovered that poor families with health...
3829,t_3895,@@37984,migrant,ca,"She married at 19 , to Milan ( Emil ) Badovina..."
3830,t_3896,@@9691377,immigrant,us,The United Kingdom is n't going to devolve int...


In [105]:
official_test_dataset =  Dataset.from_dict({"data": test_df["text"]}).with_format("torch")

In [106]:

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
tokenized_official_test_dataset = official_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3832 [00:00<?, ? examples/s]

In [107]:
predictions = trainer.predict(tokenized_official_test_dataset)
print(predictions.metrics)

predictions = predictions.predictions.argmax(axis=1)
predictions

{'test_runtime': 8.985, 'test_samples_per_second': 426.49, 'test_steps_per_second': 26.711}


array([0, 0, 0, ..., 0, 0, 0])

In [108]:
labels2file([[k] for k in predictions], 'test_roberta.txt')

## Deberta

In [124]:
from transformers import DebertaForSequenceClassification, DebertaTokenizer, AdamW
import torch

In [112]:
model_name = 'microsoft/deberta-base'
model = DebertaForSequenceClassification.from_pretrained(model_name)
tokenizer = DebertaTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [113]:
def tokenize_function(example):
    return tokenizer(example["data"], truncation=True)


tokenized_train_dataset_De = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset_De = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset_De = test_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/6700 [00:00<?, ? examples/s]

Map:   0%|          | 0/1675 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

In [114]:
batch_size = 8

training_args1 = TrainingArguments("train_logs", evaluation_strategy="epoch",\
                                  learning_rate=2e-6,
                                  num_train_epochs=10,
                                  lr_scheduler_type="cosine",
                                  optim = "adamw_torch",
                                  weight_decay=0,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  run_name="Deberta-ms",  # name of the W&B run (optional)
                                  logging_steps=1,  # how often to log to W&B
                                                                  )


trainer = Trainer(
    model,
    training_args1,
    train_dataset=tokenized_train_dataset_De,
    eval_dataset=tokenized_val_dataset_De,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [115]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.062400,0.267809,0.902090,0.035294
2,0.003800,0.311294,0.917015,0.359447
3,0.002600,0.347189,0.915821,0.331754
4,0.979800,0.330924,0.913433,0.530744
5,0.002500,0.351307,0.918209,0.486891
6,0.578600,0.396710,0.906866,0.518519
7,0.001500,0.411155,0.915821,0.518771
8,0.078500,0.421893,0.915821,0.515464
9,0.014100,0.426674,0.913433,0.518272
10,0.743700,0.427426,0.913433,0.518272


Checkpoint destination directory train_logs/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-3500 already exists and is non-empty. Saving will proceed but sa

TrainOutput(global_step=8380, training_loss=0.18782558105392816, metrics={'train_runtime': 1063.1593, 'train_samples_per_second': 63.02, 'train_steps_per_second': 7.882, 'total_flos': 4350167222584464.0, 'train_loss': 0.18782558105392816, 'epoch': 10.0})

In [116]:
predictions_val_deberta = trainer.predict(tokenized_val_dataset_De).predictions
print(predictions_val_deberta.shape)
predictions_val_deberta_lables = predictions_val_deberta.argmax(axis=1)
predictions_val_deberta_lables

(1675, 2)


array([0, 0, 0, ..., 0, 0, 0])

In [117]:
trainer.predict(tokenized_val_dataset_De).metrics

{'test_loss': 0.42742595076560974,
 'test_accuracy': 0.9134328358208955,
 'test_f1': 0.5182724252491694,
 'test_runtime': 6.4052,
 'test_samples_per_second': 261.506,
 'test_steps_per_second': 32.786}

In [118]:
predictions_test_deberta = trainer.predict(tokenized_test_dataset_De)
print(predictions_test_deberta.metrics)

predictions_test_deberta_labels = predictions_test_deberta.predictions.argmax(axis=1)
predictions_test_deberta

{'test_loss': 0.33760780096054077, 'test_accuracy': 0.9269340974212035, 'test_f1': 0.573816155988858, 'test_runtime': 8.2861, 'test_samples_per_second': 252.714, 'test_steps_per_second': 31.619}


PredictionOutput(predictions=array([[ 3.612649  , -3.882932  ],
       [ 3.4469032 , -3.7734578 ],
       [ 3.458856  , -3.7614644 ],
       ...,
       [ 3.7614148 , -3.995854  ],
       [ 0.44891518, -0.6926834 ],
       [ 3.779851  , -3.9199865 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 1, 0]), metrics={'test_loss': 0.33760780096054077, 'test_accuracy': 0.9269340974212035, 'test_f1': 0.573816155988858, 'test_runtime': 8.2861, 'test_samples_per_second': 252.714, 'test_steps_per_second': 31.619})

In [119]:
labels2file([[k] for k in predictions_val_deberta_lables], 'val_deberta.txt')

In [120]:
labels2file([[k] for k in predictions_test_deberta_labels], 'dev_deberta.txt')

In [121]:
tokenized_official_test_dataset = official_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3832 [00:00<?, ? examples/s]

In [122]:
predictions = trainer.predict(tokenized_official_test_dataset)
print(predictions.metrics)

predictions = predictions.predictions.argmax(axis=1)
predictions

{'test_runtime': 14.8147, 'test_samples_per_second': 258.662, 'test_steps_per_second': 32.333}


array([0, 0, 0, ..., 0, 0, 0])

In [123]:
labels2file([[k] for k in predictions], 'test_deberta.txt')

## Bert-Uncased Model

In [125]:
# Load model
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [126]:
def tokenize_function(example):
    return tokenizer(example["data"], truncation=True)


tokenized_train_dataset_Uc = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset_Uc = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset_Uc = test_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/6700 [00:00<?, ? examples/s]

Map:   0%|          | 0/1675 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

In [127]:
batch_size = 16

training_args1 = TrainingArguments("train_logs", evaluation_strategy="epoch",\
                                  learning_rate=2e-6,
                                  num_train_epochs=10,
                                  lr_scheduler_type="cosine",
                                  optim = "adamw_torch",
                                  weight_decay=0,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  run_name="Bert_UC",  # name of the W&B run (optional)
                                  logging_steps=1,  # how often to log to W&B
                                                                  )


trainer = Trainer(
    model,
    training_args1,
    train_dataset=tokenized_train_dataset_Uc,
    eval_dataset=tokenized_val_dataset_Uc,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [128]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.277200,0.268369,0.901493,0.023669
2,0.098600,0.244759,0.911642,0.345133
3,0.022400,0.257571,0.908657,0.246305
4,0.164600,0.241092,0.904478,0.466667
5,0.107700,0.246519,0.917612,0.477273
6,0.046700,0.257223,0.906269,0.491909
7,0.158300,0.262740,0.917015,0.483271
8,0.026900,0.265679,0.915224,0.485507
9,0.521600,0.266401,0.911045,0.473498
10,0.066200,0.266851,0.913433,0.480287


Checkpoint destination directory train_logs/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-3500 already exists and is non-empty. Saving will proceed but sa

TrainOutput(global_step=4190, training_loss=0.18687925619586662, metrics={'train_runtime': 594.8712, 'train_samples_per_second': 112.629, 'train_steps_per_second': 7.044, 'total_flos': 4517789487161280.0, 'train_loss': 0.18687925619586662, 'epoch': 10.0})

In [129]:
predictions_val_uc = trainer.predict(tokenized_val_dataset_Uc).predictions
print(predictions_val_uc.shape)
predictions_val_uc_lables = predictions_val_uc.argmax(axis=1)
predictions_val_uc_lables

(1675, 2)


array([0, 0, 0, ..., 0, 0, 0])

In [130]:
trainer.predict(tokenized_val_dataset_Uc).metrics

{'test_loss': 0.2668505907058716,
 'test_accuracy': 0.9134328358208955,
 'test_f1': 0.4802867383512544,
 'test_runtime': 4.0716,
 'test_samples_per_second': 411.389,
 'test_steps_per_second': 25.789}

In [131]:
predictions_test_uc = trainer.predict(tokenized_test_dataset_Uc)
print(predictions_test_uc.metrics)

predictions_test_uc_labels = predictions_test_uc.predictions.argmax(axis=1)
predictions_test_uc

{'test_loss': 0.24040652811527252, 'test_accuracy': 0.9183381088825215, 'test_f1': 0.5155807365439093, 'test_runtime': 5.1558, 'test_samples_per_second': 406.142, 'test_steps_per_second': 25.408}


PredictionOutput(predictions=array([[ 2.4236228, -2.5333693],
       [ 2.4641333, -2.5835   ],
       [ 2.176901 , -2.2444527],
       ...,
       [ 2.5220652, -2.554876 ],
       [-0.729819 ,  0.9270372],
       [ 2.4433508, -2.5369325]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 1, 0]), metrics={'test_loss': 0.24040652811527252, 'test_accuracy': 0.9183381088825215, 'test_f1': 0.5155807365439093, 'test_runtime': 5.1558, 'test_samples_per_second': 406.142, 'test_steps_per_second': 25.408})

In [132]:
labels2file([[k] for k in predictions_val_uc_lables], 'val_uc.txt')

In [133]:
labels2file([[k] for k in predictions_test_uc_labels], 'dev_uc.txt')

In [134]:
tokenized_official_test_dataset = official_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3832 [00:00<?, ? examples/s]

In [135]:
predictions = trainer.predict(tokenized_official_test_dataset)
print(predictions.metrics)

predictions = predictions.predictions.argmax(axis=1)
predictions

{'test_runtime': 9.7297, 'test_samples_per_second': 393.846, 'test_steps_per_second': 24.667}


array([0, 0, 0, ..., 0, 0, 0])

In [136]:
labels2file([[k] for k in predictions], 'test_uc.txt')

## Ensemble Scores

In [147]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Load labels from text files
with open('val_uc.txt', 'r') as file:
    labels_model1 = [int(line.strip()) for line in file]

with open('val_roberta.txt', 'r') as file:
    labels_model2 = [int(line.strip()) for line in file]

with open('val_deberta.txt', 'r') as file:
    labels_model3 = [int(line.strip()) for line in file]


labels_model1= val_dataset['label']

# Hard voting ensemble
ensemble_predictions_val = np.array([np.argmax(np.bincount(votes)) for votes in zip(labels_model1, labels_model2, labels_model3)])


accuracy = accuracy_score(labels_model1, ensemble_predictions_val)
precision = precision_score(labels_model1, ensemble_predictions_val)
recall = recall_score(labels_model1, ensemble_predictions_val)
f1 = f1_score(labels_model1, ensemble_predictions_val)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.9409
Precision: 0.7698
Recall: 0.5808
F1 Score: 0.6621


In [148]:
labels2file([[k] for k in ensemble_predictions_val], 'val.txt')

In [138]:
# Dev Set Results

with open('dev_roberta.txt', 'r') as file:
    labels_model1 = [int(line.strip()) for line in file]

with open('dev_deberta.txt', 'r') as file:
    labels_model2 = [int(line.strip()) for line in file]

with open('dev_uc.txt', 'r') as file:
    labels_model3 = [int(line.strip()) for line in file]


labels_model1= test_dataset['label']

# Hard voting ensemble
ensemble_predictions = np.array([np.argmax(np.bincount(votes)) for votes in zip(labels_model1, labels_model2, labels_model3)])

accuracy = accuracy_score(labels_model1, ensemble_predictions)
precision = precision_score(labels_model1, ensemble_predictions)
recall = recall_score(labels_model1, ensemble_predictions)
f1 = f1_score(labels_model1, ensemble_predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.9484
Precision: 0.8013
Recall: 0.6080
F1 Score: 0.6914


In [139]:
ensemble_predictions

array([0, 0, 0, ..., 0, 1, 0])

In [140]:
labels2file([[k] for k in ensemble_predictions], 'dev.txt')

In [141]:
# Test Set Results

with open('test_roberta.txt', 'r') as file:
    labels_model1 = [int(line.strip()) for line in file]

with open('test_deberta.txt', 'r') as file:
    labels_model2 = [int(line.strip()) for line in file]

with open('test_uc.txt', 'r') as file:
    labels_model3 = [int(line.strip()) for line in file]

# Hard voting ensemble
ensemble_predictions = np.array([np.argmax(np.bincount(votes)) for votes in zip(labels_model1, labels_model2, labels_model3)])



In [142]:
ensemble_predictions.shape

(3832,)

In [143]:
ensemble_predictions

array([0, 0, 0, ..., 0, 0, 0])

In [144]:
labels2file([[k] for k in ensemble_predictions], 'test.txt')

In [145]:
import shutil

# Replace 'folder_to_zip' with the path of the folder you want to zip
folder_path = '/content/final_data'

# Replace 'output.zip' with the desired name for the zip file
zip_filename = '/content/final_data.zip'

# Create a zip file
shutil.make_archive(zip_filename, 'zip', folder_path)

print(f"Folder '{folder_path}' zipped successfully as '{zip_filename}'")

Folder '/content/final_data' zipped successfully as '/content/final_data.zip'
